In [1]:
# -*- coding: utf-8 -*-
"""
red_mercado.py

Esta red neuronal predice dos eventos en un mercado simulado:
1. subida_brusca_precio  (subida abrupta de precios)
2. escasez

Se ha mejorado la legibilidad de las predicciones y se añaden métricas de desempeño.
"""

import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.metrics import (
    confusion_matrix,
    classification_report,
    roc_auc_score,
    accuracy_score,
    precision_score,
    recall_score
)
from keras import Sequential, Input
from keras.layers import Dense, Dropout, Normalization
from keras.metrics import Recall, Precision
import random
from keras.callbacks import EarlyStopping
import tensorflow as tf





2025-06-09 15:23:56.506217: I tensorflow/core/platform/cpu_feature_guard.cc:210] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: SSE4.1 SSE4.2 AVX AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.


In [2]:
# 1. Carga de datos
# ----------------
datos = pd.read_csv("simulacionEconomica.csv")


In [3]:
# 2. Separar atributos y objetivo
# --------------------------------
atributos = datos.loc[:, 'demanda_preajuste':'volatilidad_precio']
objetivo = datos.loc[:, 'subida_brusca_precio':'escasez']



In [4]:
# 3. Dividir en entrenamiento y prueba
# ------------------------------------
X_train, X_test, y_train, y_test = train_test_split(
    atributos, objetivo,
    test_size=0.2,
    random_state=42
)



In [5]:
# 4. Normalización
# ----------------
normalizador = Normalization()
normalizador.adapt(X_train.to_numpy())
np.mean(normalizador(X_train), axis=0)
np.var(normalizador(X_train), axis=0)

array([0.99999344, 1.000012  , 1.0000024 , 1.0000124 , 0.99998164],
      dtype=float32)

In [ ]:
# 2. Espacio de búsqueda 
"""
Esto es la busqueda aleatoria, tarda unos minutos.
 Luego, hemos entrenado la red con lo que fue el mejor parametro que nos dio.
"""
param_space = {
    'n_hidden':   [32, 64, 96],
    'dropout':    [0.1, 0.2, 0.3],
    'learning_rate': [1e-2, 1e-3, 1e-4]
}

def build_model(n_hidden, dropout_rate, lr):
    m = Sequential([
        Input(shape=(5,)),
        normalizador,
        Dense(n_hidden, activation='relu'),
        Dropout(dropout_rate),
        Dense(n_hidden//2, activation='relu'),
        Dense(2, activation='sigmoid')
    ])
    m.compile(
        optimizer=tf.keras.optimizers.Adam(learning_rate=lr),
        loss='binary_crossentropy',
        metrics=['binary_accuracy', Precision()]
    )
    return m

early_stop = EarlyStopping(
    monitor='val_precision',    # la métrica de validación que queremos maximizar
    mode='max',                 # le decimos que debe buscar el valor máximo
    patience=3,
    restore_best_weights=True
)

# 4. Bucle de búsqueda aleatoria
best_score = 0.0
best_params = None
for i in range(10):   
    # muestreo aleatorio
    hp = { k: random.choice(v) for k,v in param_space.items() }
    print(f"\n>>> Iteración {i+1}: {hp}")
    
    # crear y entrenar
    model = build_model(hp['n_hidden'], hp['dropout'], hp['learning_rate'])
    hist = model.fit(
        X_train, y_train,
        validation_data=(X_test, y_test),
        epochs=30,
        batch_size=64,
        callbacks=[early_stop],
        verbose=0
    )
    
    val_metrics = model.evaluate(X_test, y_test, verbose=0)

    val_precision = val_metrics[2]
    print(f"Precision en val: {val_precision:.4f}")
    
    if val_precision > best_score:
        best_score = val_precision
        best_params = hp

print("\n=== Mejores parámetros encontrados ===")
print(best_params, "→ Precision:", best_score)


>>> Iteración 1: {'n_hidden': 64, 'dropout': 0.3, 'learning_rate': 0.001}


/home/miguel/miniconda3/envs/BAP3/lib/python3.9/site-packages/keras/src/callbacks/early_stopping.py:153: UserWarning: Early stopping conditioned on metric `val_precision` which is not available. Available metrics are: binary_accuracy,loss,precision_13,val_binary_accuracy,val_loss,val_precision_13
  current = self.get_monitor_value(logs)


Precision en val: 0.8772

>>> Iteración 2: {'n_hidden': 64, 'dropout': 0.1, 'learning_rate': 0.0001}


/home/miguel/miniconda3/envs/BAP3/lib/python3.9/site-packages/keras/src/callbacks/early_stopping.py:153: UserWarning: Early stopping conditioned on metric `val_precision` which is not available. Available metrics are: binary_accuracy,loss,precision_14,val_binary_accuracy,val_loss,val_precision_14
  current = self.get_monitor_value(logs)


Precision en val: 0.8832

>>> Iteración 3: {'n_hidden': 64, 'dropout': 0.3, 'learning_rate': 0.0001}


...

Precision en val: 0.8797

>>> Iteración 9: {'n_hidden': 32, 'dropout': 0.1, 'learning_rate': 0.001}


/home/miguel/miniconda3/envs/BAP3/lib/python3.9/site-packages/keras/src/callbacks/early_stopping.py:153: UserWarning: Early stopping conditioned on metric `val_precision` which is not available. Available metrics are: binary_accuracy,loss,precision_21,val_binary_accuracy,val_loss,val_precision_21
  current = self.get_monitor_value(logs)


Precision en val: 0.8654

>>> Iteración 10: {'n_hidden': 32, 'dropout': 0.3, 'learning_rate': 0.01}


/home/miguel/miniconda3/envs/BAP3/lib/python3.9/site-packages/keras/src/callbacks/early_stopping.py:153: UserWarning: Early stopping conditioned on metric `val_precision` which is not available. Available metrics are: binary_accuracy,loss,precision_22,val_binary_accuracy,val_loss,val_precision_22
  current = self.get_monitor_value(logs)


Precision en val: 0.8594

=== Mejores parámetros encontrados ===
{'n_hidden': 64, 'dropout': 0.3, 'learning_rate': 0.0001} → Precision: 0.9007114171981812


In [6]:
# 5. Definición de la red
# -----------------------
# obtuvimos esto => {'n_hidden': 64, 'dropout': 0.3, 'learning_rate': 0.0001} → Precision: 0.9007114171981812
red_mercado = Sequential([
    Input(shape=(5,)),         # 5 variables de entrada
    normalizador,              # normaliza cada batch
    Dense(64, activation='relu'),  # capa oculta amplia
    Dropout(0.3),                  # evita sobreajuste
    Dense(32, activation='relu'),  # capa oculta más pequeña
    Dense(2, activation='sigmoid') # 2 salidas binarias
])



In [7]:
# 6. Compilación
# --------------
red_mercado.compile(
    optimizer=tf.keras.optimizers.Adam(learning_rate=0.0001),
    loss='binary_crossentropy',
    metrics=['binary_accuracy', Precision()]
)



In [8]:
# 7. Entrenamiento
# ----------------
historial = red_mercado.fit(
    X_train, y_train,
    batch_size=128,
    epochs=30,
    validation_split=0.2,
    verbose=1
)



Epoch 1/30
175/175 ━━━━━━━━━━━━━━━━━━━━ 2s 5ms/step - binary_accuracy: 0.7612 - loss: 0.6393 - precision: 0.6516 - val_binary_accuracy: 0.8342 - val_loss: 0.5514 - val_precision: 0.8308
Epoch 2/30
175/175 ━━━━━━━━━━━━━━━━━━━━ 1s 3ms/step - binary_accuracy: 0.8288 - loss: 0.5279 - precision: 0.8509 - val_binary_accuracy: 0.8509 - val_loss: 0.4425 - val_precision: 0.8435
Epoch 3/30
175/175 ━━━━━━━━━━━━━━━━━━━━ 1s 4ms/step - binary_accuracy: 0.8488 - loss: 0.4274 - precision: 0.8406 - val_binary_accuracy: 0.8746 - val_loss: 0.3637 - val_precision: 0.8543
Epoch 4/30
175/175 ━━━━━━━━━━━━━━━━━━━━ 1s 4ms/step - binary_accuracy: 0.8681 - loss: 0.3598 - precision: 0.8572 - val_binary_accuracy: 0.8776 - val_loss: 0.3194 - val_precision: 0.8521
Epoch 5/30
175/175 ━━━━━━━━━━━━━━━━━━━━ 1s 4ms/step - binary_accuracy: 0.8740 - loss: 0.3201 - precision: 0.8525 - val_binary_accuracy: 0.8794 - val_loss: 0.2942 - val_precision: 0.8483
Epoch 6/30
175/175 ━━━━━━━━━━━━━━━━━━━━ 1s 3ms/step - binary_accuracy:

In [9]:
# 8. Evaluación en conjunto de prueba
# -----------------------------------
y_pred_prob = red_mercado.predict(X_test)
# Probabilidades a etiquetas 0/1 con umbral 0.5
y_pred = (y_pred_prob >= 0.5).astype(bool)

# Métricas globales (promedio macro)
accuracy = accuracy_score(y_test, y_pred)
precision = precision_score(y_test, y_pred, average='macro', zero_division=0)
recall = recall_score(y_test, y_pred, average='macro', zero_division=0)
roc_auc = roc_auc_score(y_test, y_pred_prob, average='macro')

print(f"Accuracy (macro): {accuracy:.4f}")
print(f"Precision (macro): {precision:.4f}")
print(f"Recall (macro): {recall:.4f}")
print(f"ROC AUC (macro): {roc_auc:.4f}")

# Reporte por clase
print("\nClassification Report:")
print(classification_report(y_test, y_pred, target_names=['subida_brusca', 'escasez']))

# Matrices de confusión
from sklearn.metrics import multilabel_confusion_matrix

# y_test e y_pred tienen forma (n_muestras, 2)
cms = multilabel_confusion_matrix(y_test, y_pred)

# cms[0]  → matriz para la etiqueta “subida brusca”
# cms[1]  → matriz para la etiqueta “escasez”

print("Matriz de confusión — subida brusca:")
print(cms[0])
print("\nMatriz de confusión — escasez:")
print(cms[1])




219/219 ━━━━━━━━━━━━━━━━━━━━ 1s 2ms/step
Accuracy (macro): 0.7924
Precision (macro): 0.8928
Recall (macro): 0.8086
ROC AUC (macro): 0.9369

Classification Report:
               precision    recall  f1-score   support

subida_brusca       0.82      0.65      0.72      2833
      escasez       0.96      0.97      0.97      1164

    micro avg       0.87      0.74      0.80      3997
    macro avg       0.89      0.81      0.85      3997
 weighted avg       0.86      0.74      0.79      3997
  samples avg       0.27      0.27      0.27      3997

Matriz de confusión — subida brusca:
[[3767  400]
 [1002 1831]]

Matriz de confusión — escasez:
[[5795   41]
 [  34 1130]]


/home/miguel/miniconda3/envs/BAP3/lib/python3.9/site-packages/sklearn/metrics/_classification.py:1509: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in samples with no predicted labels. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/home/miguel/miniconda3/envs/BAP3/lib/python3.9/site-packages/sklearn/metrics/_classification.py:1509: UndefinedMetricWarning: Recall is ill-defined and being set to 0.0 in samples with no true labels. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/home/miguel/miniconda3/envs/BAP3/lib/python3.9/site-packages/sklearn/metrics/_classification.py:1509: UndefinedMetricWarning: F-score is ill-defined and being set to 0.0 in samples with no true nor predicted labels. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is

In [10]:
# 9. Lectura fácil de predicciones nuevas
# ---------------------------------------
# Tomamos 10 muestras de ejemplo para ver probabilidades y clases
muestra = datos.iloc[1510:1520]
prob_muestra = red_mercado.predict(muestra.loc[:, 'demanda_preajuste':'volatilidad_precio'])
pred_muestra = (prob_muestra >= 0.5).astype(bool)

# Construimos DataFrame con los resultados
df_pred = pd.DataFrame({
    'demanda_preajuste': muestra.demanda_preajuste,
    'volatilidad_precio': muestra.volatilidad_precio,
    'prob_subida_brusca': prob_muestra[:, 0],
    'prob_escasez': prob_muestra[:, 1],
    'pred_subida_brusca': pred_muestra[:, 0],
    'pred_escasez': pred_muestra[:, 1]
})

print("\nPredicciones de ejemplo:")
print(df_pred.to_string(index=False))


1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 42ms/step

Predicciones de ejemplo:
 demanda_preajuste  volatilidad_precio  prob_subida_brusca  prob_escasez  pred_subida_brusca  pred_escasez
            2427.0             1.34536            0.671062      0.009368                True         False
            2487.0             1.46544            0.451639      0.001092               False         False
            2519.0             1.53623            0.447828      0.001001               False         False
            2528.0             1.56445            0.358579      0.000265               False         False
            2617.0             1.63936            0.499381      0.001413               False         False
            2688.0             1.82962            0.717124      0.016006                True         False
            2676.0             2.00250            0.601822      0.002260                True         False
            2693.0             2.01184            0.163984      0.000011        